In [ ]:
from src.modules import NPKRecommender, SeedTreatmentRecommender, CoverCropMixRecommender, FinalRecommeder

In [2]:
import json
import pandas as pd

In [3]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

In [4]:
with open("prompts//npk_recommend_prompt.json", "r", encoding="utf-8") as f:
    npk_recommend_prompt_data = json.load(f)
npk_recommend_system_prompt = npk_recommend_prompt_data["system_prompt"]
npk_recommend_user_prompt = npk_recommend_prompt_data["user_prompt"]

with open("prompts//seed_treatment_prompt.json", "r", encoding="utf-8") as f:
    seed_treatment_prompt_data = json.load(f)
seed_treatment_system_prompt = seed_treatment_prompt_data["system_prompt"]
seed_treatment_user_prompt = seed_treatment_prompt_data["user_prompt"]

with open("prompts//final_recommend_prompt.json", "r", encoding="utf-8") as f:
    final_recommend_prompt_data = json.load(f)
final_recommend_system_prompt = final_recommend_prompt_data["system_prompt"]
final_recommend_user_prompt = final_recommend_prompt_data["user_prompt"]


In [5]:
cover_crop_mix_recommender_ = CoverCropMixRecommender("data/target_cover_crop.csv", "data/cover_crop_goal.json", "data/recommend_cover_crop.csv")
npk_recommender_ = NPKRecommender("data/target_npk.csv", npk_recommend_system_prompt, npk_recommend_user_prompt, TOGETHER_API_KEY)
seed_treatment_recommender_ = SeedTreatmentRecommender("data/target_seed_treatment.csv", seed_treatment_system_prompt, seed_treatment_user_prompt, TOGETHER_API_KEY)
final_recommender_ = FinalRecommeder(final_recommend_system_prompt, final_recommend_user_prompt, TOGETHER_API_KEY)

In [ ]:
inputs = {
    "current_cash_crop": "Maize",
    "next_cash_crop": "Soybean",
    "after_next_cash_crop": "Potato",
    "crop_type": "Maize",
    "N": 65,      
    "P": 18,       
    "K": 160,      
    "pH": 5.8,
    "CEC": 12.4,
    "SOC": 1.2,
    "sand": 45,  
    "silt": 35,
    "clay": 20
}

In [ ]:
npk_recommend = npk_recommender_.recommend(
    crop_type=inputs["crop_type"],
    N=inputs["N"],
    P=inputs["P"],
    K=inputs["K"],
    sand=inputs["sand"],
    silt=inputs["silt"],
    clay=inputs["clay"],
    ph=inputs["pH"],
    cec=inputs["CEC"],
    soc=inputs["SOC"]
)
print(npk_recommend)

### Reasoning Section
1. The current levels of N, P, and K are 65 mg/kg, 18 mg/kg, and 160 mg/kg, respectively. For maize, the optimal ranges are typically around 80-120 mg/kg for N, 20-30 mg/kg for P, and 150-200 mg/kg for K. Thus, N is slightly below the optimal range, P is at the lower end of the optimal range, and K is within the optimal range.
2. The pH of 5.8 is slightly acidic, which can affect nutrient uptake. Most nutrients are available to plants when the pH is between 6.0 and 7.0. At a pH of 5.8, the availability of nutrients like P, K, and some micronutrients might be slightly reduced, but N availability is generally less affected by slightly acidic conditions.
3. The CEC of 12.4 cmol/kg is moderate, indicating the soil has a moderate capacity to retain nutrients. This means the soil can hold a reasonable amount of nutrients but might not retain them as well as soils with higher CEC values.
4. The SOC of 1.2% is relatively low, which can impact the soil's ability to retain 

In [8]:
npk_recommend.split("### Conclusion Section")[-1]

'\n[(N_rec, 60.0, Increased due to slightly low initial level and soil characteristics prone to leaching), (P_rec, 0, Already at the lower end of the optimal range and considering soil conditions), (K_rec, 0, Within the optimal range and soil retains it moderately well)]'

In [ ]:
seed_treatment_recommend = seed_treatment_recommender_.recommend(
    crop_type=inputs["crop_type"],
    N=inputs["N"],
    P=inputs["P"],
    K=inputs["K"],
    sand=inputs["sand"],
    silt=inputs["silt"],
    clay=inputs["clay"],
    ph=inputs["pH"],
    cec=inputs["CEC"],
    soc=inputs["SOC"]
)
print(seed_treatment_recommend)

### Reasoning Section
1. Analyzing the nutrient levels: 
   - N (65 mg/kg) is relatively low for maize, which requires a significant amount of nitrogen for growth.
   - P (18 mg/kg) is low, as maize benefits from higher phosphorus levels, especially during early growth stages.
   - K (160 mg/kg) is medium to high, which is suitable for maize as it requires a substantial amount of potassium.

2. Evaluating pH effects:
   - The pH of 5.8 is slightly acidic, which can affect microbial activity. Most microbes thrive in a pH range of 6.0 to 8.0. However, some microbes like Mycorrhiza can tolerate a wider pH range.

3. Considering CEC:
   - A CEC of 12.4 cmol/kg indicates moderate nutrient retention capacity. This is not extremely low, suggesting that the soil can retain some nutrients, but it may not be optimal for all inoculants.

4. Assessing SOC:
   - SOC of 1.2% is relatively low, indicating limited organic matter and potentially reduced microbial support capacity. This suggests the nee

In [ ]:
cover_crop_mix_recommend = cover_crop_mix_recommender_.recommend(
    current_cash_crop=inputs["current_cash_crop"],
    next_cash_crop=inputs["next_cash_crop"],
    after_next_cash_crop=inputs["after_next_cash_crop"],
    N=inputs["N"],
    P=inputs["P"],
    K=inputs["K"],
    sand=inputs["sand"],
    silt=inputs["silt"],
    clay=inputs["clay"]
)
print(cover_crop_mix_recommend)

[('Rye', ['Erosion Control', 'Weed Suppression']), ('Mustard', ['Pest/Disease Management', 'Weed Suppression']), ('Hairy Vetch', ['Build Soil Organic Matter'])]


In [17]:
npk_recommend.split("### Conclusion Section\n")[-1]

'[(N_rec, 60.0, Increased due to slightly low initial level and soil characteristics prone to leaching), (P_rec, 0, Already at the lower end of the optimal range and considering soil conditions), (K_rec, 0, Within the optimal range and soil retains it moderately well)]'

In [12]:
new_inputs = {
    "crop_type": inputs["crop_type"],
    "NPK_recs": npk_recommend.split("### Conclusion Section")[-1],
    "inoculant_methods": seed_treatment_recommend.split("### Conclusion Section")[-1],
    "cover_crops": cover_crop_mix_recommend,
}

In [18]:
a= inputs["crop_type"]
b= npk_recommend.split("### Conclusion Section\n")[-1]
c= seed_treatment_recommend.split("### Conclusion Section\n")[-1]
d= cover_crop_mix_recommend

In [21]:
b

'[(N_rec, 60.0, Increased due to slightly low initial level and soil characteristics prone to leaching), (P_rec, 0, Already at the lower end of the optimal range and considering soil conditions), (K_rec, 0, Within the optimal range and soil retains it moderately well)]'

In [23]:
e = final_recommender_.recommend(a,b,c,d)
print(e)

**Farmer-Friendly Summary for Maize Crop**

### Fertilizer Advice

Based on the soil analysis, here are the fertilizer recommendations for your maize crop:

* **Nitrogen (N):** An additional application of 60.0 kg/ha is recommended. This is because the initial soil level is slightly low and the soil characteristics are prone to leaching, which could lead to nitrogen deficiency during the crop's growth.
* **Phosphorus (P):** No additional application is needed, as the current level is already at the lower end of the optimal range and considering the soil conditions.
* **Potassium (K):** No additional application is needed, as the current level is within the optimal range and the soil retains it moderately well.

### Inoculants

The following inoculant-method pairs are recommended for your maize crop:

* **Mycorrhiza - Powder mixing:** This method is suitable for low phosphorus conditions and is tolerant of slightly acidic pH. Mycorrhiza helps in improving phosphorus uptake by the plants